In [0]:
import mlflow
from mlflow import MlflowClient

# --- KONFIGURATION ---
CATALOG = "workspace"
SCHEMA  = "default"
DRY_RUN = False   

# Username holen (für Experiment-Pfade)
try:
    current_user = spark.sql("SELECT current_user()").first()[0]
except:
    current_user = "unknown"

print(f"🧹 STARTE CLEANUP für User: {current_user}")
print(f"📍 Ziel: {CATALOG}.{SCHEMA}")
print(f"⚠️ Modus: {'DRY RUN (nur Simulation)' if DRY_RUN else 'DESTRUCTIVE (Löschen)'}")
print("-" * 60)

# Init
spark.sql(f"USE CATALOG `{CATALOG}`")
spark.sql(f"USE SCHEMA `{SCHEMA}`")
client = MlflowClient()

def run_sql(sql):
    if DRY_RUN:
        print(f"[SQL Dry] {sql}")
    else:
        try:
            spark.sql(sql)
            print(f"✅ {sql}")
        except Exception as e:
            print(f"⚠️ Fehler bei: {sql} -> {e}")

def pick_name(row, prefs=("name","tableName","viewName","function","functionName","volume")):
    d = row.asDict()
    for k in prefs:
        if k in d: return d[k]
    return row[1]

# ---------------------------------------------------------
# 1. MLflow MODELLE löschen
# ---------------------------------------------------------
print("\n--- 1. MLflow Modelle bereinigen ---")
try:
    print("   ⏳ Frage Liste aller Modelle ab (das kann kurz dauern)...")
    # Wir holen einfach die ersten 1000 Modelle ohne Filter (stabiler)
    all_models = client.search_registered_models(max_results=1000)
    
    # Wir filtern selbst in Python, das ist oft schneller als Server-Side
    models_to_delete = [m for m in all_models if m.name.startswith(f"{CATALOG}.{SCHEMA}.")]
    
    print(f"   🔎 Habe {len(models_to_delete)} Modelle zum Löschen gefunden.")

    for model in models_to_delete:
        if DRY_RUN:
            print(f"[MLflow Dry] Würde Modell löschen: {model.name}")
        else:
            print(f"   🗑️ Lösche Modell '{model.name}'...")
            try:
                # 1. Versionen löschen
                versions = client.search_model_versions(f"name='{model.name}'")
                for v in versions:
                    client.delete_model_version(name=model.name, version=v.version)
                    print(f"      - Version {v.version} gelöscht.")
                
                # 2. Modell löschen
                client.delete_registered_model(name=model.name)
                print(f"      ✅ Modell entfernt.")
            except Exception as e:
                print(f"      ⚠️ Fehler beim Löschen von {model.name}: {e}")

except Exception as e:
    print(f"⚠️ KRITISCHER FEHLER beim Abrufen der Modelle: {e}")
    print("   -> Überspringe MLflow-Cleanup und mache mit SQL weiter...")

# ---------------------------------------------------------
# 2. MLflow EXPERIMENTE löschen
# ---------------------------------------------------------
print("\n--- 2. MLflow Experimente bereinigen ---")
# Wir suchen Experimente im Home-Ordner des Users, die nach unseren Kurs-Namen klingen
experiments = mlflow.search_experiments(filter_string=f"name LIKE '/Users/{current_user}/%'")

# Liste der bekannten Kurs-Experimente (Sicherheitsfilter, damit wir nicht alles löschen)
target_experiments = ["House_Price_Predictor", "Dashboard_Priority_Predictor", "Course_Price_Prediction"]

for exp in experiments:
    # Prüfen, ob der Experiment-Name einen unserer Kurs-Namen enthält
    if any(target in exp.name for target in target_experiments):
        if DRY_RUN:
            print(f"[MLflow Dry] Würde Experiment löschen: {exp.name}")
        else:
            try:
                client.delete_experiment(exp.experiment_id)
                print(f"✅ Experiment '{exp.name}' gelöscht.")
            except Exception as e:
                print(f"⚠️ Fehler bei Experiment {exp.name}: {e}")

# ---------------------------------------------------------
# 3. SQL Objekte (Dein Code - Optimiert)
# ---------------------------------------------------------
print("\n--- 3. SQL Objekte (Views, Tabellen, Funktionen) ---")

# Views
df_views = spark.sql(f"SHOW VIEWS IN `{CATALOG}`.`{SCHEMA}`")
for r in df_views.collect():
    v = pick_name(r)
    run_sql(f"DROP VIEW IF EXISTS `{CATALOG}`.`{SCHEMA}`.`{v}`")

# Tabellen
df_tables = spark.sql(f"SHOW TABLES IN `{CATALOG}`.`{SCHEMA}`")
for r in df_tables.collect():
    t = pick_name(r)
    run_sql(f"DROP TABLE IF EXISTS `{CATALOG}`.`{SCHEMA}`.`{t}`")

# Funktionen
df_funcs = spark.sql(f"SHOW USER FUNCTIONS IN `{CATALOG}`.`{SCHEMA}`")
for r in df_funcs.collect():
    f = pick_name(r)
    run_sql(f"DROP FUNCTION IF EXISTS `{CATALOG}`.`{SCHEMA}`.`{f}`")

# ---------------------------------------------------------
# 4. VOLUMES & DATEIEN (Der physische Müll)
# ---------------------------------------------------------
print("\n--- 4. Volumes und Dateien ---")

# Zuerst den Inhalt der Volumes löschen (Checkpoints, JSONs, CSVs)
# Sonst bleibt der Müll auf der Festplatte, auch wenn das Volume-Objekt weg ist.
df_vols = spark.sql(f"SHOW VOLUMES IN `{CATALOG}`.`{SCHEMA}`")
vols = [pick_name(r) for r in df_vols.collect()]

for v in vols:
    path = f"/Volumes/{CATALOG}/{SCHEMA}/{v}"
    if DRY_RUN:
        print(f"[FS Dry] Würde Inhalt löschen von: {path}")
        print(f"[SQL Dry] Würde Volume droppen: {v}")
    else:
        try:
            # Inhalt löschen (rekursiv)
            dbutils.fs.rm(path, True)
            print(f"✅ Dateien in '{path}' gelöscht.")
            # Volume Objekt löschen
            spark.sql(f"DROP VOLUME IF EXISTS `{CATALOG}`.`{SCHEMA}`.`{v}`")
            print(f"✅ Volume '{v}' gedropped.")
        except Exception as e:
             print(f"⚠️ Fehler bei Volume {v}: {e}")

# ---------------------------------------------------------
# 5. Checkpoints im DBFS (Falls wir /tmp genutzt haben)
# ---------------------------------------------------------
if not DRY_RUN:
    # Optional: Alte Checkpoints aus früheren Lektionen im DBFS root
    try:
        dbutils.fs.rm("/tmp/checkpoints", True)
        print("\n✅ /tmp/checkpoints bereinigt.")
    except: pass

print("-" * 60)
print("🎉 CLEANUP COMPLETE!")

🧹 STARTE CLEANUP für User: philippe.christen@fhnw.ch
📍 Ziel: workspace.default
⚠️ Modus: DESTRUCTIVE (Löschen)
------------------------------------------------------------

--- 1. MLflow Modelle bereinigen ---
   ⏳ Frage Liste aller Modelle ab (das kann kurz dauern)...
   🔎 Habe 0 Modelle zum Löschen gefunden.

--- 2. MLflow Experimente bereinigen ---

--- 3. SQL Objekte (Views, Tabellen, Funktionen) ---

--- 4. Volumes und Dateien ---


{"ts": "2025-11-26 11:10:58.959", "level": "ERROR", "logger": "pyspark.sql.connect.client.logging", "msg": "GRPC Error received", "context": {}, "exception": {"class": "_MultiThreadedRendezvous", "msg": "<_MultiThreadedRendezvous of RPC that terminated with:\n\tstatus = StatusCode.INTERNAL\n\tdetails = \"Public DBFS root is disabled. Access is denied on path: /tmp/checkpoints\"\n\tdebug_error_string = \"UNKNOWN:Error received from peer  {grpc_message:\"Public DBFS root is disabled. Access is denied on path: /tmp/checkpoints\", grpc_status:13, created_time:\"2025-11-26T11:10:58.959103472+00:00\"}\"\n>", "stacktrace": [{"class": null, "method": "_execute_and_fetch_as_iterator", "file": "/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py", "line": "1935"}, {"class": null, "method": "__next__", "file": "<frozen _collections_abc>", "line": "356"}, {"class": null, "method": "send", "file": "/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/cl

------------------------------------------------------------
🎉 CLEANUP COMPLETE!
